In [18]:
#using bertopic with my data
from bertopic import BERTopic
import pandas as pd
from umap import UMAP
from nltk.corpus import stopwords
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [19]:
#import my data
df = pd.read_excel("Mentor_CSAT.xlsx")
df.head()

,Unnamed: 0,activityid,Lastdate,SentDate,StudentName,PMName,PMOperationsID,PathID,PipePathwayID,StudentType,...,"To help us better understand your experience, please explain the reasons for your satisfaction ratin",My Mentor shares useful information.,My Mentor responds to me within 24 hours.,My Mentor helps me get questions and issues resolved.,My Mentor cares about me and my success.,Sent Date,Semester,% Communicated,% Not Communicated,"Overall, how satisfied are you with BYU-Pathway Mentoring?"
0,0,F22B8E8C-476D-EE11-8DF0-00224808045A,2023-10-17 23:47:33,10/16/2023,Hnin Wai Oo,Maria Carmela Villenas Gaela,10260,22822379,22822379,Matriculation,...,She's really kind to me,Mostly true,Moderately true,Slightly true,Moderately true,2023-10-16,Fall 2023,Communicated,NaN,Satisfied
1,1,C049BF3C-0F10-EE11-8F6D-0022480A40EB,2023-06-21 08:40:19,6/20/2023,Adrian James Oriewo,Susan Kapiya Chitembo,10121,22560197,22560197,Online Degree - BYU-Idaho,...,The Peer Mentor always encourages me to keep o...,Definitely true,Definitely true,Definitely true,Definitely true,2023-06-20,Spring 2023,Communicated,NaN,Very satisfied
2,2,15D312C2-80F9-ED11-8849-0022480AE0F6,2023-05-23 15:44:56,5/22/2023,Marie Angelene Empleo Diaz,Julie Anne Gollayan Alberto Espino,10503,22525995,22525995,PathwayConnect,...,She replies whenever I need her help.,Moderately true,Moderately true,Moderately true,Moderately true,2023-05-22,Spring 2023,Communicated,NaN,Very satisfied
3,3,8F082B81-0C25-EE11-9CBE-0022480AE0F6,2023-07-18 01:43:35,7/17/2023,Gabart Eugenio Jean Louis,Michael Aboagye Adu,10389,20062864,20062864,PathwayConnect,...,Best mentor ever thus far,Definitely true,Definitely true,Definitely true,Definitely true,2023-07-17,Spring 2023,Communicated,NaN,Very satisfied
4,4,264787A9-EC98-EE11-BE37-0022480AE1C8,2023-12-12 12:47:48,12/11/2023,Nomathamsanqa Maho,Evans Nana Baah Ankomah,10243,13242085,13242085,Online Degree - Ensign College,...,NaN,NaN,NaN,NaN,NaN,2023-12-11,Fall 2023,NaN,Not Communicated,NaN


In [20]:
data =  df['To help us better understand your experience, please explain the reasons for your satisfaction ratin'].values
stopwords = stopwords.words('english')

In [21]:
def lemmatization(texts, allowed_posttags = ["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(str(text))
        new_text = []
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        if final != '':
            texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0:5])

['really kind', 'always encourage keep push foward study most importantly share scripture strengthen', 'reply need help', 'good mentor ever thus far', 'very satisfied peer mentor always available need always answer question very important']


In [22]:
#bert using umap
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=100)
topic_model = BERTopic(embedding_model="paraphrase-multilingual-MiniLM-L12-v2",umap_model=umap_model,language='english', calculate_probabilities=True, nr_topics=10, top_n_words=10)

In [23]:
#bert using embeddings
# topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")


In [24]:
topics, probs = topic_model.fit_transform(lemmatized_texts)

In [25]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,16238,-1_help_mentor_always_peer,"[help, mentor, always, peer, very, question, n...",[help lot write very well always good message ...
1,0,12298,0_mentor_help_always_peer,"[mentor, help, always, peer, very, need, encou...","[mentor always there help reach, love fact alw..."
2,1,4221,1_email_contact_time_respond,"[email, contact, time, respond, communicate, s...","[send great email respond question, contact ti..."
3,2,3171,2_always_check_answer_question,"[always, check, answer, question, help, concer...","[always answer question, always answer questio..."
4,3,1176,3_good_satisfied_very_excellent,"[good, satisfied, very, excellent, great, amaz...","[very good, very good, very good]"
5,4,177,4_certificate_register_registration_help,"[certificate, register, registration, help, ti...","[able get certificate, help ask certificate kn..."
6,5,122,5_pathway_connect_portal_help,"[pathway, connect, portal, help, journey, answ...","[always communicate help pathway journey, easy..."
7,6,26,6_none_bad_awsome_negative,"[none, bad, awsome, negative, but, suck, neutr...","[none, none, none]"
8,7,22,7_text_texte_always_often,"[text, texte, always, often, ahve, dot, help, ...","[text regularly, always answer text help, text]"
9,8,21,8_zoom_meeting_question_help,"[zoom, meeting, question, help, screen, meet, ...",[send email check question set zoom meeting ex...


In [26]:
topic_model.get_topic(0)

[('mentor', 0.05602944858643826),
 ('help', 0.04949021802094684),
 ('always', 0.04625661073002377),
 ('peer', 0.03731080785474958),
 ('very', 0.03583961324823161),
 ('need', 0.03299860337742911),
 ('encourage', 0.026347358426618214),
 ('good', 0.02597155955046057),
 ('helpful', 0.02576077194739277),
 ('give', 0.024058083578059962)]

In [27]:
topic_model.get_representative_docs(0)

['mentor always there help reach',
 'love fact always ready willing help struggle new know go thing peer mentor always there answer question offer encouragement week write just check make sure okay share inspire quote church leader great encouragement mention struggle financially other thing time talk say thing sound know go reason give know circumstance provide opportunity educate always make way ask more peer mentoring amazing very grateful peer mentor',
 'peer mentor supportive encourage always answer question']

In [28]:
# df = pd.DataFrame({"topic": topics, "document":data})
# df

In [29]:
#Generate topic labels
topic_labels = topic_model.generate_topic_labels(nr_words=3, topic_prefix = False, word_length=15, separator= " - ")
topic_model.set_topic_labels(topic_labels)
topic_model.get_topic_info().head(10)

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,16238,-1_help_mentor_always_peer,help - mentor - always,"[help, mentor, always, peer, very, question, n...",[help lot write very well always good message ...
1,0,12298,0_mentor_help_always_peer,mentor - help - always,"[mentor, help, always, peer, very, need, encou...","[mentor always there help reach, love fact alw..."
2,1,4221,1_email_contact_time_respond,email - contact - time,"[email, contact, time, respond, communicate, s...","[send great email respond question, contact ti..."
3,2,3171,2_always_check_answer_question,always - check - answer,"[always, check, answer, question, help, concer...","[always answer question, always answer questio..."
4,3,1176,3_good_satisfied_very_excellent,good - satisfied - very,"[good, satisfied, very, excellent, great, amaz...","[very good, very good, very good]"
5,4,177,4_certificate_register_registration_help,certificate - register - registration,"[certificate, register, registration, help, ti...","[able get certificate, help ask certificate kn..."
6,5,122,5_pathway_connect_portal_help,pathway - connect - portal,"[pathway, connect, portal, help, journey, answ...","[always communicate help pathway journey, easy..."
7,6,26,6_none_bad_awsome_negative,none - bad - awsome,"[none, bad, awsome, negative, but, suck, neutr...","[none, none, none]"
8,7,22,7_text_texte_always_often,text - texte - always,"[text, texte, always, often, ahve, dot, help, ...","[text regularly, always answer text help, text]"
9,8,21,8_zoom_meeting_question_help,zoom - meeting - question,"[zoom, meeting, question, help, screen, meet, ...",[send email check question set zoom meeting ex...


In [30]:
#Things I can do
#Change n-grams
# topic_model.update_topics(data, n_gram_range=(1, 3))

#Merge specific topic
# topic_model.merge_topics(data,topics_to_merge=[1, 8, 12])

#Reduce the number of topics
# topic_model.reduce_topics(data, nr_topics=100)

#Find specific topics
# topic_model.find_topics("decision trees", top_n=1)

In [31]:
topic_model.visualize_topics()

In [32]:
topic_model.visualize_barchart()

In [33]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [34]:
topic_model.visualize_heatmap()

In [35]:
topic_model.visualize_distribution(topic_model.probabilities_[0], min_probability=0.015)

In [36]:
#predict with model
num_of_topics = 3
# similar_topics, similarity = topic_model.find_topics(data,top_n=num_of_topics)
#print it

In [37]:
#S